## TU-E2210 Financial Engineering I
Exercise 6 solutions
Kaila, Toepfer
Python version 
3.11.2022

If you don't have ```pyfinance``` installed, run the following:

In [2]:
import sys
!{sys.executable} -m pip install pyfinance

     |████████████████████████████████| 57 kB 1.8 MB/s eta 0:00:011


These are the packages used in this exercise set.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from pyfinance.options import BSM

These exercises are partly done with R. Remember that you can use R only for computation. Before writing code, you must understand each problem thoroughly.

1. A portfolio containing derivative securities on only one asset has Delta 5000 and Gamma -200. A call on the asset with $\Delta (C)=0.4$ and $\Gamma (C)=0.05$, and a put on the same asset, with $\Delta (P)=-0.5$ and $\Gamma (P)=0.07$, are currently traded. How do you make the portfolio Delta-neutral and Gamma-neutral?

### Solution

Take positions of size $x_1$ and $x_2$, respectively, in the call and put options specified above. The value $\Pi$ of the new portfolio is $\Pi = V+x_1C +x_2P$, where V is the value of the original portfolio. This portfolio will be Delta- and Gamma-neutral, provided that $x_1$ and $x_2$ are chosen such that $$\begin{aligned}
\Delta (\Pi) & = \Delta (V) + x_1\Delta (C)+x_2\Delta (P)= 5000+0.4x_1-0.5x_2=0\nonumber\\
\Gamma (\Pi) & = \Gamma (V) +x_1\Gamma (C)+x_2\Gamma(P) =-200+0.05x_1+0.07x_2=0.
\end{aligned}$$ The solution to this linear system is$$x_1=-\frac{250000}{53}= -4716.98\mathrm{\ \ and\ \ }x_2=\frac{330000}{53}=6226.42.$$ To make the initial portfolio as close to Delta- and Gamma-neutral as possible by only trading in the given call and put options, 4717 calls must be sold and 6226 put options must be bought. The Delta and Gamma of the new portfolios are $$\begin{aligned}
\Delta (\Pi) & = \Delta (V) -4717\Delta (C)+6226\Delta (P)= 0.02\nonumber\\
\Gamma (\Pi) & = \Gamma (V) -4717\Gamma (C)+6226\Gamma(P) =-0.03.
\end{aligned}$$ To understand how well hedged the portfolio $\Pi$ is, recall that the initial portfolio had $\Delta (V)= 5000$ and $\Gamma (V)=-200$.

We write $0.4x_1-0.5x_2=-5000$ and $0.05x_1+0.07x_2=200$. We then denote by matrix $A$ the left-side constants, by vector $x$ the variables $x_1$ and $x_2$, and by vector $b$ the right-side constants, so that we have an equation $Ax=b$.

With ```Python```: In the console, use ```np.linalg.solve```.

*Delta* and *Gamma* for call and put options

In [2]:
A = [ [0.4, -0.5], [0.05,  0.07] ]

*Delta* and *Gamma* for the initial portfolios

In [3]:
b = [ -5000, 200 ]

Solution for ```Ax = b```

In [4]:
X = np.linalg.solve(A, b)
print(X)

[-4716.98113208  6226.41509434]


2.  You are long 1000 call options with strike 90 and maturity of three months. Assume that the underlying asset has a lognormal distribution with drift μ = 0.08 $\mu=0.08$ and volatility $\sigma=0.2$ and that the spot price of the asset is 92. The risk-free rate is $r=0.05$. What Delta-hedging position do you need to take? Notice that maturities are given in years so 3 months is $T=0.25$.

    ### Solution

    A long call position is Delta-hedged by a short position in the underlying asset. Delta-hedging the long position in 1000 calls is done by shorting$$1000\Delta (C)=1000\mathcal{N}(d_1)=653.5$$ units of the underlying asset, where $$\begin{aligned}
    d_1 & = \frac{\ln (S/T)+(r+\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}}\mathrm{\ \ and}\nonumber\\
    \mathcal{N} (z) & = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^z e^{-y^2/2}dy.
    \end{aligned}$$ with $S=92$, $E=90$, $T=0.25$, $\sigma=0.2$, and $r=0.05$.

    With `Python`: The Delta of the Call option can be computed by typing `norm.cdf(d1)`, where

In [5]:
S = 92      # Spot price
E = 90      # Strike price
T = 3/12    # 3 months
t = 0       # Current time = 0
sigma = 0.2 # Volatility
r = 0.05    # Risk-free rate
eps = np.finfo(float).eps

We use the first term from the Black-Scholes formula:

In [6]:
d1 = (np.log(S/E) + (r + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t) + eps)

Normal cumulative distribution function.

This is the *delta* for one call options.

In [7]:
Nd1 = norm.cdf(d1)

print('Delta: %f \n'%(Nd1))

Delta: 0.653501 



Alternative way: use the ```BSM``` function

In [8]:
delta_alt =  BSM(kind='call', S0=S, K=E, T=(T - t), r=r, sigma=sigma).delta() 

print('Delta with blsdelta: %.4f \n' %(delta_alt))

print("Required number of options: %.0f \n"%(np.round(1000*Nd1)))

Delta with blsdelta: 0.6535 

Required number of options: 654 



Note that for Delta-hedging purposes (or even for pricing options in the Black-Scholes model), it is not necessary to know the drift $\mu$ of the underlying asset, since $\Delta (C)$ does not depend on $\mu$.

3.  Solve with `Python` the vega-delta hedging example in the end of Lecture 6.

4.  You buy $1000$ six-month ATM (at-the-money) call options on a nondividend-paying asset that follows a lognormal process with volatility $0.3$. The spot price is $100$. Assume that the risk-free interest rate is $r=0.05$.

    a)  How much money do you pay for the options?

    b)  How much do you need to buy/short the underlying asset to delta-hedge the call position?

    c)  Next trading day, the asset opens at 98. What is the value of your position now (the option and shares position)? Recall that there are 252 trading days in one year.

    d)  Had you not delta-hedged your position, how much would you have lost due to the fall in the price of the underlying asset?

    ### Solution

    a)  Using the Black-Scholes formula with inputs $S_1=E=100$, $T=0.5$, $\sigma =0.3$, $r=0.05$, we find that the value of one call option is $C_1=9.634877$. Therefore, 9634.88 must be paid for 1000 calls. With `Python`, the call price can be computed with the function `BSM;`.

In [9]:
S = 100
E = 100 #ATM
sigma = 0.3
dividend = 0
r = 0.05
T = 6/12 # this is in years
N = 1000;

 Then the option price is

In [10]:
d1 = (np.log(S/E) + (r + sigma ** 2 / 2) * T) /(sigma * np.sqrt(T) + eps)
d2 = d1 - sigma * np.sqrt(T)
C = norm.cdf(d1) * S - E * np.exp(-r * T) * norm.cdf(d2)

Cdirectly = BSM(kind='call', S0=S, K=E, T=T, r=r, sigma=sigma).value()

print('Price per option: %.3f \nTo pay: %.2f \n'%( C, 1000*C) )

Price per option: 9.635 
To pay: 9634.88 



b)  To delta-hedge the long call position you need to short$$1000\Delta(C)=1000\mathcal{N}(d_1)=589$$units of the underlying. Recall the `BSM` function to solve this with `Python`.

Then we can calculate (remember to round-off in the end, one cannot trade fractions of options):

In [11]:
delta_pf = 1000 * BSM(kind='call', S0=S, K=E, T=T, r=r, sigma=sigma).delta()

nHedge = np.round(delta_pf)

print('Stocks required to delta-hedge: %d \n'%(nHedge))
print('Value of position: %.2f \n'%( N * C - nHedge * S))

Stocks required to delta-hedge: 589 

Value of position: -49265.12 



c)  The new spot price and maturity are S2 = 98 $S_2=98$ and T2 = 125/252 $T_2=125/252$. The value of the call option is $C_2=8.453134$ and therefore the value of the position is$$1000C_2-589S_2=-49268.89.$$Recalling that there are 252 trading days in one year and notating the portfolio as `N` call options and `Nhedge` assets:

In [12]:
S1 = 98 # new spot
T1 = T - 1/252 #maturity diminishes

C1 = BSM(kind='call', S0=S1, K=E, T=T1, r=r, sigma=sigma).value()


   
print('New value of position: %.2f \n'%( N * C1 - nHedge * S1))

New value of position: -49268.88 



d)  If the long call position is not delta-hedged, the loss due to the fall in the spot price of the underlying asset is $$1000(C_2-C_1)=-1181.75.$$ For the delta-hedged portfolio, the loss incurred is $$(1000C_2-589S_2)-(1000C_1-589S_1)=-3.75.$$ As expected, the loss is much smaller when the call position is delta-hedged.

In R, we can calculate this: Had you not delta-hedged your position, how much would you have lost due to the fall in the price of the underlying asset?

In [13]:
 N * (C1 - C)

-1181.7533164310703

In [14]:
print('Loss without hedging: %.2f \n'%(N * (C1 - C)))
print('Loss with hedging:    %.2f \n' %((N * C1 - nHedge * S1) - (N * C - nHedge * S)))

Loss without hedging: -1181.75 

Loss with hedging:    -3.75 



5.  Black-Scholes greeks, vega

    The Black-Scholes formula for call options is given by $$\begin{aligned}
    C(t,S,E,T,\sigma) & = S\mathcal{N}(d_+)-Ee^{-r(T-t)}\mathcal{N}(d_-),\mathrm{\ \ }S=S_t\nonumber\\
    d_{\pm} & = \frac{\ln (S/T)+(r\pm\sigma^2/2)(T-t)}{\sigma\sqrt{T-t}}\mathrm{\ \ and}\nonumber\\
    \mathcal{N} (z) & = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^z e^{-y^2/2}dy.
    \end{aligned}$$ Show that$$Ee^{-r(T-t)}\mathcal{N}'(d_-)=S\mathcal{N}'(d_+).$$You will need $$\mathcal{N}'(x)=\frac{\exp (-x^2/2)}{\sqrt{2\pi}}$$ How is this result related to vega, $\kappa$, the sensitivity of Black-Scholes prices with respect to the volatility?

    ### Solution

    $$
    \begin{aligned}
    Ee^{-r(T-t)}\mathcal{N}'(d_-) & = Ee^{-r(T-t)}\frac{e^{-d^2_-/2}}{\sqrt{2\pi}}\nonumber\\
     & = Ee^{-r(T-t)}\frac{e^{-(d_+-\sigma\sqrt{T-t})^2/2}}{\sqrt{2\pi}} \nonumber\\
     & = Ee^{-r(T-t)}e^{\sigma\sqrt{T-t}d_+}e^{-\sigma^2(T-t)/2}\mathcal{N}'(d_+) \nonumber\\
     & = Ee^{-r(T-t)}\frac{S}{E}e^{(r+\sigma^2/2)(T-t)}e^{-\sigma^2(T-t)/2}\mathcal{N}'(d_+) \nonumber\\ 
     & = S\mathcal{N}'(d_+) .
    \end{aligned}
    $$ The vega of a European call is $\kappa=S\mathcal{N}'(d_+)\sqrt{t}=Ee^{-r(T-t)}\mathcal{N}'(d_-)\sqrt{t}$.

6.  Black-Scholes Greeks, delta

    The delta in the Black-Scholes model is the partial derivative of the option price with respect to the underlying asset: $\Delta:=\partial C(t,S)/\partial S$. Show that$$\Delta =\mathcal{N}(d_+).$$ Remember that $d_+$ is a function of $S$ and of $t$.

    ### Solution

    Now $$\frac{\partial}{\partial S}d_{\pm}=\frac{1}{S\sigma\sqrt{T-t}}.$$Hence, using the partial derivative above and the equality in a), we get$$\begin{aligned}
    \frac{\partial}{\partial S}C(t,S) & =\mathcal{N}(d_+)+S\mathcal{N}'(d_+)\frac{\partial}{\partial S}d_+-Ee^{-r(T-t)}\mathcal{N}'(d_-)\frac{\partial}{\partial S}d_-\nonumber\\
    & = \mathcal{N}(d_+) +\frac{\mathcal{N}'(d_+)}{\sigma\sqrt{T-t}}-\frac{\mathcal{N}'(d_+)}{\sigma\sqrt{T-t}}\nonumber\\
    & = \mathcal{N}(d_+).
    \end{aligned}$$

7.  EXTRA: Taking positions

    Copy the file `FEexercise7_2022Data.xls` from the course homepage (this was the file from Exercise 5). Based on the S&P100 option prices, try to make portfolios such as butterfly, bull spread, bear spread, etc. You can then follow how the value of the portfolio changes as a function of time to maturity.

    You can simply draw three figures, one for the price of the underlying, one for the separate prices of the options and one for the value of the portfolio. Remember that you can take either a long or a short position regarding the portfolio (if you assume that the price of the underlying will move a little, you can take a long position in a butterfly; if you assume that the price of the underlying will move a lot, you can short a butterfly.) Try different strike prices when taking positions and look how it affects the value of the corresponding portfolio.

    The exercises have partly been taken from Stefanica, Dan. *A Primer for the Mathematics of Financial Engineering*. Fe Press, 2011.


------------------------------------------------